In [2]:
# ============================================================================
# NOTEBOOK: 05_transformaciones_normalizacion.ipynb
# Objetivo: Transformar variables para mejorar normalidad
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import PowerTransformer
import warnings
warnings.filterwarnings('ignore')

# Cargar dataset
df = pd.read_csv('PCOS_data_espanol.csv')
print(f"Dataset cargado: {df.shape}")

# =============================================================================
# 1. CLASIFICAR VARIABLES POR SKEWNESS
# =============================================================================
target = 'SOP (S/N)'
numeric_vars = df.select_dtypes(include=[np.number]).columns.drop(target).tolist()

# Calcular skewness
skewness_data = []
for var in numeric_vars:
    skew = stats.skew(df[var].dropna())
    skewness_data.append({'Variable': var, 'Skewness': skew, 'Abs_Skewness': abs(skew)})

df_skew = pd.DataFrame(skewness_data).sort_values('Abs_Skewness', ascending=False)

# Clasificar
criticas = df_skew[df_skew['Abs_Skewness'] > 5]['Variable'].tolist()
severas = df_skew[(df_skew['Abs_Skewness'] > 3) & (df_skew['Abs_Skewness'] <= 5)]['Variable'].tolist()
moderadas = df_skew[(df_skew['Abs_Skewness'] > 1) & (df_skew['Abs_Skewness'] <= 3)]['Variable'].tolist()
aceptables = df_skew[df_skew['Abs_Skewness'] <= 1]['Variable'].tolist()

print("\n" + "="*80)
print("CLASIFICACIÓN POR SKEWNESS")
print("="*80)
print(f"🔴 CRÍTICAS (>5): {len(criticas)} variables")
print(criticas)
print(f"\n🟠 SEVERAS (3-5): {len(severas)} variables")
print(severas)
print(f"\n🟡 MODERADAS (1-3): {len(moderadas)} variables")
print(moderadas)
print(f"\n✅ ACEPTABLES (<1): {len(aceptables)} variables")
print(f"Total: {len(aceptables)} variables NO necesitan transformación")

# =============================================================================
# 2. APLICAR TRANSFORMACIONES
# =============================================================================
df_transformed = df.copy()

# Variables a transformar
vars_to_transform = criticas + severas + moderadas
print(f"\n🔧 Variables a transformar: {len(vars_to_transform)}")

# Diccionario para guardar qué transformación se aplicó
transformations_applied = {}

# ---------------------------
# 2A. YEO-JOHNSON (para críticas y severas)
# ---------------------------
vars_yeo_johnson = criticas + severas
print(f"\nAplicando Yeo-Johnson a {len(vars_yeo_johnson)} variables...")

for var in vars_yeo_johnson:
    try:
        data = df[var].values.reshape(-1, 1)
        pt = PowerTransformer(method='yeo-johnson', standardize=False)
        transformed = pt.fit_transform(data)
        df_transformed[var] = transformed.flatten()
        transformations_applied[var] = 'Yeo-Johnson'
    except:
        print(f"  ⚠️ Error en {var}, se mantiene original")
        transformations_applied[var] = 'Error - Original'

# ---------------------------
# 2B. LOG(x+1) (para moderadas)
# ---------------------------
print(f"\nAplicando Log(x+1) a {len(moderadas)} variables...")

for var in moderadas:
    try:
        df_transformed[var] = np.log1p(df[var])
        transformations_applied[var] = 'Log(x+1)'
    except:
        print(f"  ⚠️ Error en {var}, se mantiene original")
        transformations_applied[var] = 'Error - Original'

# ---------------------------
# 2C. MANTENER (aceptables)
# ---------------------------
for var in aceptables:
    transformations_applied[var] = 'Sin transformación'

print("\n✅ Transformaciones completadas")

# =============================================================================
# 3. COMPARAR MÉTRICAS ANTES VS DESPUÉS
# =============================================================================
print("\n" + "="*80)
print("COMPARACIÓN: ANTES vs DESPUÉS")
print("="*80)

comparison = []

for var in numeric_vars:
    # ANTES
    skew_before = stats.skew(df[var].dropna())
    kurt_before = stats.kurtosis(df[var].dropna())
    shapiro_p_before = stats.shapiro(df[var].dropna()).pvalue if len(df[var].dropna()) < 5000 else np.nan
    
    # DESPUÉS
    skew_after = stats.skew(df_transformed[var].dropna())
    kurt_after = stats.kurtosis(df_transformed[var].dropna())
    shapiro_p_after = stats.shapiro(df_transformed[var].dropna()).pvalue if len(df_transformed[var].dropna()) < 5000 else np.nan
    
    # Mejora
    mejora_skew = abs(skew_before) - abs(skew_after)
    mejora_percent = (mejora_skew / abs(skew_before) * 100) if abs(skew_before) > 0 else 0
    
    comparison.append({
        'Variable': var,
        'Transformacion': transformations_applied[var],
        'Skew_Antes': skew_before,
        'Skew_Despues': skew_after,
        'Kurt_Antes': kurt_before,
        'Kurt_Despues': kurt_after,
        'Shapiro_P_Antes': shapiro_p_before,
        'Shapiro_P_Despues': shapiro_p_after,
        'Mejora_Skew': mejora_skew,
        'Mejora_Percent': mejora_percent,
        'Normal_Antes': shapiro_p_before > 0.05 if not np.isnan(shapiro_p_before) else False,
        'Normal_Despues': shapiro_p_after > 0.05 if not np.isnan(shapiro_p_after) else False
    })

df_comparison = pd.DataFrame(comparison)

# Ordenar por mejora
df_comparison_sorted = df_comparison.sort_values('Mejora_Percent', ascending=False)

print("\nTOP 20 MAYORES MEJORAS:")
print(df_comparison_sorted.head(20)[['Variable', 'Transformacion', 'Skew_Antes', 
                                      'Skew_Despues', 'Mejora_Percent']])

# Contar normales
normales_antes = df_comparison['Normal_Antes'].sum()
normales_despues = df_comparison['Normal_Despues'].sum()

print(f"\n📊 RESUMEN:")
print(f"Variables normales ANTES: {normales_antes}/{len(numeric_vars)} ({normales_antes/len(numeric_vars)*100:.1f}%)")
print(f"Variables normales DESPUÉS: {normales_despues}/{len(numeric_vars)} ({normales_despues/len(numeric_vars)*100:.1f}%)")
print(f"Mejora: +{normales_despues - normales_antes} variables")

# Guardar comparación
df_comparison_sorted.to_csv('comparacion_transformaciones.csv', index=False)
print("\n✅ Guardado: comparacion_transformaciones.csv")

# =============================================================================
# 4. VISUALIZACIONES: ANTES VS DESPUÉS
# =============================================================================
print("\n" + "="*80)
print("GENERANDO VISUALIZACIONES")
print("="*80)

# Solo las que fueron transformadas
vars_transformadas = [v for v in vars_to_transform if transformations_applied[v] != 'Error - Original']

for i, var in enumerate(vars_transformadas):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'{var}\nTransformación: {transformations_applied[var]}', 
                 fontsize=14, fontweight='bold')
    
    # ANTES - Histograma
    axes[0, 0].hist(df[var].dropna(), bins=30, edgecolor='black', alpha=0.7, color='crimson')
    axes[0, 0].set_title(f'ANTES - Histograma\nSkew: {stats.skew(df[var].dropna()):.2f}')
    axes[0, 0].axvline(df[var].mean(), color='blue', linestyle='--', linewidth=2, label='Media')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # ANTES - Q-Q Plot
    stats.probplot(df[var].dropna(), dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title('ANTES - Q-Q Plot')
    axes[0, 1].grid(alpha=0.3)
    
    # DESPUÉS - Histograma
    axes[1, 0].hist(df_transformed[var].dropna(), bins=30, edgecolor='black', 
                    alpha=0.7, color='green')
    axes[1, 0].set_title(f'DESPUÉS - Histograma\nSkew: {stats.skew(df_transformed[var].dropna()):.2f}')
    axes[1, 0].axvline(df_transformed[var].mean(), color='blue', linestyle='--', 
                       linewidth=2, label='Media')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # DESPUÉS - Q-Q Plot
    stats.probplot(df_transformed[var].dropna(), dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title('DESPUÉS - Q-Q Plot')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'trans_{i+1:02d}_{var.replace("/", "_").replace(" ", "_")}.png', 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    if (i+1) % 5 == 0:
        print(f"  Procesadas: {i+1}/{len(vars_transformadas)}")

print(f"\n✅ Guardadas {len(vars_transformadas)} visualizaciones")

# =============================================================================
# 5. GRÁFICO RESUMEN: MEJORA EN SKEWNESS
# =============================================================================
# Solo variables que mejoraron
df_mejoras = df_comparison_sorted[df_comparison_sorted['Mejora_Percent'] > 0].head(15)

fig, ax = plt.subplots(figsize=(12, 8))
x = np.arange(len(df_mejoras))
width = 0.35

bars1 = ax.barh(x - width/2, df_mejoras['Skew_Antes'].abs(), width, 
                label='Antes', color='crimson', alpha=0.7)
bars2 = ax.barh(x + width/2, df_mejoras['Skew_Despues'].abs(), width, 
                label='Después', color='green', alpha=0.7)

ax.set_yticks(x)
ax.set_yticklabels(df_mejoras['Variable'], fontsize=9)
ax.set_xlabel('|Skewness|', fontsize=12, fontweight='bold')
ax.set_title('TOP 15 MEJORAS EN NORMALIDAD (|Skewness|)', 
             fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('resumen_mejoras_skewness.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Guardado: resumen_mejoras_skewness.png")

# =============================================================================
# 6. GUARDAR DATASET TRANSFORMADO
# =============================================================================
df_transformed.to_csv('PCOS_data_transformado.csv', index=False)
print("\n✅ Dataset transformado guardado: PCOS_data_transformado.csv")

# =============================================================================
# 7. REPORTE FINAL
# =============================================================================
print("\n" + "="*80)
print("REPORTE FINAL")
print("="*80)

# Crear reporte
reporte = {
    'Metrica': [
        'Variables totales',
        'Variables críticas (>5)',
        'Variables severas (3-5)',
        'Variables moderadas (1-3)',
        'Variables aceptables (<1)',
        '',
        'Transformaciones Yeo-Johnson',
        'Transformaciones Log(x+1)',
        'Sin transformación',
        '',
        'Normales antes',
        'Normales después',
        'Variables que mejoraron',
        'Promedio mejora skewness (%)'
    ],
    'Valor': [
        len(numeric_vars),
        len(criticas),
        len(severas),
        len(moderadas),
        len(aceptables),
        '',
        len(vars_yeo_johnson),
        len(moderadas),
        len(aceptables),
        '',
        normales_antes,
        normales_despues,
        (df_comparison['Mejora_Skew'] > 0).sum(),
        df_comparison[df_comparison['Mejora_Skew'] > 0]['Mejora_Percent'].mean()
    ]
}

df_reporte = pd.DataFrame(reporte)
print(df_reporte.to_string(index=False))

df_reporte.to_csv('reporte_transformaciones.csv', index=False)
print("\n✅ Guardado: reporte_transformaciones.csv")

print("\n" + "="*80)
print("ARCHIVOS GENERADOS:")
print("="*80)
print("✅ PCOS_data_transformado.csv (dataset final)")
print("✅ comparacion_transformaciones.csv (métricas detalladas)")
print("✅ reporte_transformaciones.csv (resumen ejecutivo)")
print(f"✅ {len(vars_transformadas)} visualizaciones individuales")
print("✅ resumen_mejoras_skewness.png (gráfico comparativo)")
print("\n🎯 SIGUIENTE PASO: 06_resolucion_multicolinealidad.ipynb")

Dataset cargado: (538, 42)

CLASIFICACIÓN POR SKEWNESS
🔴 CRÍTICAS (>5): 6 variables
['Progesterona (ng/mL)', 'FSH (mUI/mL)', 'Frecuencia Cardiaca (lpm)', 'beta-HCG I (mUI/mL)', 'beta-HCG II (mUI/mL)', 'Glucosa (mg/dl)']

🟠 SEVERAS (3-5): 4 variables
['Presión Sistólica (mmHg)', 'TSH (mUI/L)', 'Presión Diastólica (mmHg)', 'Ratio FSH/LH']

🟡 MODERADAS (1-3): 10 variables
['Número Abortos', 'Prolactina (ng/mL)', 'AMH (ng/mL)', 'LH (mUI/mL)', 'Frecuencia Respiratoria (rpm)', 'Ejercicio Regular (S/N)', 'Vitamina D3 (ng/mL)', 'Años Casada', 'Tamaño Folículo Prom (I) (mm)', 'Crecimiento Vello (S/N)']

✅ ACEPTABLES (<1): 21 variables
Total: 21 variables NO necesitan transformación

🔧 Variables a transformar: 20

Aplicando Yeo-Johnson a 10 variables...

Aplicando Log(x+1) a 10 variables...

✅ Transformaciones completadas

COMPARACIÓN: ANTES vs DESPUÉS

TOP 20 MAYORES MEJORAS:
                         Variable      Transformacion  Skew_Antes  \
21                    TSH (mUI/L)         Yeo-Johns